---
title: "Post 3 - Replication Study: Dissecting Racial Bias in Algorithms"
author: Col McDermott
date: "03-12-2025"
description: "An introductory replication study of an investigating racial bias in an algorithm applied to manage population health."
format: html
code-fold: true
execute:
  warning: false
  message: false
---

# Abstract



## Accessing the Data

*The following replication study aims to replicate elements of [this original study](https://www.science.org/doi/full/10.1126/science.aax2342) while following [these guidelines](https://pmc.ncbi.nlm.nih.gov/articles/PMC10019630/)*

In [ ]:
# Includuing all additional imports
from matplotlib.colors import LinearSegmentedColormap
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import itertools

# Reading in population health data
hd = pd.read_csv("https://gitlab.com/labsysmed/dissecting-bias/-/raw/master/data/data_new.csv?inline=false")

*Including all additional imports*

Explanation of data source and features.

## Reproducing Figure 1

*Code Description*

**Figure 1 (Reproduced)**

Analysis of reproducing **Figure 1**.

## Reproducing Figure 3

*Code Description*

**Figure 3 (Reproduced)**

Analysis of reproducing **Figure 3**.

## Modeling Cost Disparity

### Data Preparation

*Code Description*

Analysis of data preparation.

### Modeling

*Code Description*

Analysis of modeling.

# Discussion

Discussion of the replication study.

*During the implementation process of this replication study, I collaborated with _____*